
<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/03_Einsum_Is_All_You_Need.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 3: Einsum (Jeden by wszystkimi rządzić)

Większość operacji w PyTorch (`sum`, `transpose`, `mm`, `bmm`) to tylko specjalne przypadki **Konwencji Sumacyjnej Einsteina**.

Funkcja `torch.einsum('wzór', a, b)` pozwala zdefiniować operację za pomocą indeksów literowych.

**Zasady Gry:**
1.  Każdy wymiar oznaczamy literą (np. `i`, `j`, `k`).
2.  **Po lewej stronie strzałki (`->`):** Nazywamy wymiary wejściowe.
3.  **Po prawej stronie strzałki:** Nazywamy wymiary wyjściowe.
    *   Jeśli litera zniknęła -> **Sumujemy** po tym wymiarze.
    *   Jeśli litery zmieniły kolejność -> **Transponujemy**.
    *   Jeśli litera się powtarza w wejściu (np. `i, i`) -> **Mnożymy** elementy (Hadamard).

To brzmi abstrakcyjnie, ale w praktyce jest genialnie proste.

In [1]:
import torch

# Dane testowe
A = torch.tensor([[1, 2, 3], 
                  [4, 5, 6]]) # (2, 3) -> i, j

B = torch.tensor([[7, 8, 9], 
                  [10, 11, 12]]) # (2, 3) -> i, j

print(f"A shape: {A.shape}")
print(f"B shape: {B.shape}")

A shape: torch.Size([2, 3])
B shape: torch.Size([2, 3])


## Poziom 1: Podstawy (Suma i Transpozycja)

Zapiszmy proste operacje w języku Einsum.

1.  **Transpozycja:** Zamieniamy wiersze (`i`) z kolumnami (`j`).
    *   Wzór: `ij -> ji`
2.  **Suma wszystkich elementów:** Wszystkie wymiary znikają.
    *   Wzór: `ij ->` (pusty wynik oznacza skalar)
3.  **Suma po kolumnach:** Sumujemy wymiar `j`, zostaje `i`.
    *   Wzór: `ij -> i`

In [2]:
# 1. Transpozycja (A.T)
# i=2, j=3 -> j=3, i=2
transposed = torch.einsum('ij -> ji', A)
print("--- Transpozycja (ij -> ji) ---")
print(transposed)

# 2. Suma całkowita (torch.sum(A))
# i, j znikają -> sumujemy po obu
total_sum = torch.einsum('ij ->', A)
print(f"\n--- Suma Całkowita (ij -> ) ---\n{total_sum}")

# 3. Suma wierszy (torch.sum(A, dim=1))
# j znika -> sumujemy po j (kolumnach), zostają wiersze i
row_sum = torch.einsum('ij -> i', A)
print(f"\n--- Suma po wierszach (ij -> i) ---\n{row_sum}")
# 1+2+3=6, 4+5+6=15

--- Transpozycja (ij -> ji) ---
tensor([[1, 4],
        [2, 5],
        [3, 6]])

--- Suma Całkowita (ij -> ) ---
21

--- Suma po wierszach (ij -> i) ---
tensor([ 6, 15])


## Poziom 2: Mnożenie Macierzy (Matrix Multiplication)

To tutaj `einsum` błyszczy. Klasyczne mnożenie macierzy $A \times C$.
*   $A$: (2, 3) -> `ik`
*   $C$: (3, 5) -> `kj`
*   Wynik: (2, 5) -> `ij`

Wymiar `k` (wewnętrzny) znika, więc po nim sumujemy. To definicja mnożenia macierzy.
Wzór: `ik, kj -> ij`

In [3]:
# Nowa macierz C (3 wiersze, 5 kolumn)
C = torch.randn(3, 5)

# Tradycyjne mnożenie (mm)
res_mm = torch.mm(A.float(), C)

# Einsum
# i=2, k=3 | k=3, j=5 -> i=2, j=5
res_ein = torch.einsum('ik, kj -> ij', A.float(), C)

print("--- Matrix Multiplication ---")
print(f"Kształt wyniku: {res_ein.shape}")

# Sprawdźmy czy to to samo
print(f"Czy identyczne? {torch.allclose(res_mm, res_ein)}")

--- Matrix Multiplication ---
Kształt wyniku: torch.Size([2, 5])
Czy identyczne? True


## Poziom 3: Batch Matrix Multiplication (BMM)

To operacja, którą wykonuje każdy Transformer (GPT) miliardy razy.
Mamy Batch (`b`) macierzy. Chcemy pomnożyć każdą macierz z batcha przez odpowiadającą jej macierz z drugiego batcha.

*   Input 1: `(b, i, k)`
*   Input 2: `(b, k, j)`
*   Output: `(b, i, j)`

Tradycyjnie: `torch.bmm`.
Einsum: `bik, bkj -> bij` (Po prostu dodajemy `b` na początku, które "przechodzi dalej").

In [4]:
batch_size = 10
i, k, j = 20, 30, 40

X = torch.randn(batch_size, i, k)
Y = torch.randn(batch_size, k, j)

# Tradycyjne BMM
res_bmm = torch.bmm(X, Y)

# Einsum BMM
# b przechodzi bez zmian. k znika (sumowanie).
res_ein_bmm = torch.einsum('bik, bkj -> bij', X, Y)

print("--- Batch Matrix Multiplication ---")
print(f"Czy identyczne? {torch.allclose(res_bmm, res_ein_bmm)}")

--- Batch Matrix Multiplication ---
Czy identyczne? True


## Boss Level: Attention Mechanism

Wzór na Attention to:
$$ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{Q K^T}{\sqrt{d_k}}\right) V $$

W kodzie PyTorch (bez einsum) to koszmar z wymiarami:
`Q` ma kształt `(Batch, Heads, Seq, Dim)`.
Żeby pomnożyć $Q$ i $K^T$, musimy robić transpozycje, uważać na wymiary głów...

Z `einsum` to jedna linijka.
*   $Q$: `bhqd` (batch, heads, query_len, dim)
*   $K$: `bhkd` (batch, heads, key_len, dim)
*   Wynik ($Q K^T$): `bhqk` (batch, heads, query_len, key_len)

Wymiar `d` znika (sumujemy po nim - iloczyn skalarny).

In [5]:
# Symulacja danych do Attention
batch = 2
heads = 4
seq_len = 8
dim = 16

Q = torch.randn(batch, heads, seq_len, dim)
K = torch.randn(batch, heads, seq_len, dim)

print(f"Q shape: {Q.shape}")
print(f"K shape: {K.shape}")

# Tradycyjnie (ból głowy)
# Musimy transponować K na (batch, heads, dim, seq_len) przed mnożeniem
scores_manual = torch.matmul(Q, K.transpose(-2, -1))

# EINSUM (Czysta poezja)
# bhqd - Q
# bhkd - K (używamy 'k' zamiast 'q' dla długości, choć tu są równe)
# Wynik ma być macierzą podobieństwa query-to-key: bhqk
scores_einsum = torch.einsum('bhqd, bhkd -> bhqk', Q, K)

print("\n--- ATTENTION SCORES ---")
print(f"Wynik shape: {scores_einsum.shape}")
print(f"Czy identyczne? {torch.allclose(scores_manual, scores_einsum)}")

# A teraz mnożenie przez V (Value)
V = torch.randn(batch, heads, seq_len, dim)

# Wynik Attention * V
# Scores: bhqk
# V:      bhkd (k to ten sam wymiar co w scores - length)
# Wynik:  bhqd (wracamy do oryginalnego kształtu)
# Sumujemy po 'k'
context = torch.einsum('bhqk, bhkd -> bhqd', scores_einsum, V)

print(f"Context shape: {context.shape}")

Q shape: torch.Size([2, 4, 8, 16])
K shape: torch.Size([2, 4, 8, 16])

--- ATTENTION SCORES ---
Wynik shape: torch.Size([2, 4, 8, 8])
Czy identyczne? True
Context shape: torch.Size([2, 4, 8, 16])


## 🥋 Black Belt Summary

Dlaczego warto używać `einsum`?
1.  **Czytelność:** Widać dokładnie, które wymiary są mnożone, a które sumowane. Nie musisz zgadywać, co robi `transpose(1, 2)`.
2.  **Bezpieczeństwo:** Nie musisz robić `reshape` ani `view`, co jest częstym źródłem błędów w wymiarach.
3.  **Wydajność:** PyTorch kompiluje `einsum` do zoptymalizowanych jąder CUDA (często łączy operacje, np. transpozycję i mnożenie w jednym kroku).

**Zasada kciuka:**
Jeśli masz w kodzie więcej niż jeden `.permute()` lub `.transpose()` przed mnożeniem -> zamień to na `einsum`.